In [1]:
import pandas as pd
import numpy as np
from sklearn import  preprocessing
import xgboost as xgb 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import geohash2
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from collections import defaultdict
from ProgressBar import ProgressBar
import gc
from math import radians, cos, sin, asin, sqrt

c:\users\roy\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
df=pd.read_csv("../train.csv")
shop=pd.read_csv("../mall.csv")
#test=pd.read_csv("../test.csv")
df=pd.merge(df,shop[['shop_id','mall_id','category_id','price']],how='left',on='shop_id')
train=df
mall_list=list(set(list(shop.mall_id)))
bssid = pd.read_csv('../data/wifi_finall.csv')

In [11]:
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 1000

In [8]:
df.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,mall_id,category_id,price
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...,m_1409,c_38,42
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...,m_1409,c_38,42
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...,m_4079,c_30,40
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...,m_6587,c_27,49
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...,m_3005,c_6,62


In [110]:
#df['time_stamp']=pd.to_datetime(df['time_stamp'])

In [9]:
shop.head()

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
1,s_133,c_6,121.134362,31.197511,58,m_6587
2,s_251,c_38,121.000505,30.907667,34,m_5892
3,s_372,c_30,119.864982,26.659876,44,m_625
4,s_456,c_26,122.594243,31.581499,44,m_3839


In [3]:
from collections import Counter

In [9]:
for mall in ["m_1293"]:
    print("------"+"training on "+mall+"-------")
    train1=train[train.mall_id==mall].reset_index(drop=True)
    bssid1 = bssid[bssid.mall_id==mall].reset_index(drop=True)
    shop1 = shop[shop.mall_id==mall].reset_index(drop=True)
    a = bssid1["bssid"].value_counts()
    b = a >= 19
    kk = pd.DataFrame(b)
    kk.columns = ["state"]
    kk["bssid"] = kk.index
    tmp = pd.DataFrame(kk)
    delate_wifi = list(tmp[tmp["state"]==False]["bssid"].unique())
    tmp1 = pd.merge(bssid1,kk,on='bssid',how='left')
    tmp_train = tmp1[tmp1["state"]==True]
    del tmp_train["state"]
    wifi_to_shops = defaultdict(lambda : defaultdict(lambda :0))
    l=[]
    ll = []
    lll= {"avg_wifi":[]}
    llll = {"num_wifi":[]}
    gg = {"num_connect":[]}
    wifi_dict = {}
    bar = ProgressBar(max_value=len(list(tmp_train["bssid"].unique())),name="wifi")
    for wifi in list(tmp_train["bssid"].unique()):
        bssid_wifi = tmp_train[tmp_train["bssid"] == "{}".format(wifi)]
        k = bssid_wifi["shop_id"].value_counts()
        wifi_to_shops[wifi]=dict(zip(list(k.index),list(k.values)))
        bar.update()
    bar = ProgressBar(max_value=train1.shape[0],name="wifi")
    for index,row in train1.iterrows():
        r = {}
        rr = {}
        wifi_list = [wifi.split('|') for wifi in row['wifi_infos'].split(';')]
        wifi = sorted([wifi.split('|') for wifi in row['wifi_infos'].split(';')],key=lambda x:int(x[1]),reverse=True)[0]
#         zz = {}
#         for wi in wifi:
#             X = Counter(wifi_to_shops[wi[0]])
#             Y = Counter(zz)
#             zz = dict(X+Y)
        #if wifi[0] not in delate_wifi:
        ll.append(wifi_to_shops[wifi[0]])
        kk = 0
        num = 0
        for i in wifi_list:
            kk += int(i[1])
            num += (i[2]=="true")
            if i not in delate_wifi:
                r[i[0]]=int(i[1])
        l.append(r)
        lll["avg_wifi"].append(kk/len(wifi_list))
        llll["num_wifi"].append(len(wifi_list))
        gg["num_connect"].append(num)   
        bar.update()
#         kk = []
#         for index1,row1 in shop1.iterrows():
#             gg = haversine(row["longitude"],row["latitude"],row1["longitude"],row1["latitude"])
#             rr[row1["shop_id"]+"new"] = gg
#             kk.append(gg)
#         kk = sorted(kk)
#         kk_new = kk[:10]
#         d1 = {k: v for k, v in rr.items() if v in kk_new}
#         lll.append(d1)
    del wifi_to_shops
    del tmp1
    del tmp_train
    gc.collect()
    train1 = pd.concat([train1,pd.DataFrame(l),pd.DataFrame(ll),pd.DataFrame(lll),pd.DataFrame(llll),pd.DataFrame(gg)], axis=1)
    del l
    del ll
    gc.collect()
#     one_hot_time_stamp = pd.get_dummies(train1['time_stamp'],prefix='time_stamp1')
#     train1 = pd.concat([train1,one_hot_time_stamp],axis=1)
#     lb = preprocessing.LabelEncoder()
#     train1["situation"] = list(map(lambda x,y:geohash2.encode(x,y,precision=9),train1["longitude"],train1["latitude"]))
#     train1["situation1"] = lb.fit_transform(train1["situation"])
#     one_hot_situation = pd.get_dummies(train1['situation'],prefix='situation1')
    #train1 = pd.concat([train1,one_hot_situation], axis=1)
    len_ = train1.shape[0]
    df_train = train1[:int(0.9 * len_)]
    df_test=train1[int(0.9 * len_):]
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train['shop_id'].values))
    df_train['label'] = lbl.transform(list(df_train['shop_id'].values))
    df_test['label'] = lbl.transform(list(df_test['shop_id'].values))  
    num_class=df_train['label'].max()+1
    feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','mall_id','wifi_infos',"situation"]]    

------training on m_1293-------

## #     wifi     # (1214 : 0 -> 1214) Task Finished. Time Cost:   0 h   0 min  22.31 s; Average:   0 h   0 min 0.01837 s  ##min 0.01839 s 

## #     wifi     # (22280 : 0 -> 22280) Task Finished. Time Cost:   0 h   0 min  48.21 s; Average:   0 h   0 min 0.002164 s  ##in 0.002164 s 


c:\users\roy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\roy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
train1.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,mall_id,b_10020256,b_10027728,b_1003266,...,s_757418,s_761178,s_774697,s_819780,s_848684,s_97281,s_97701,avg_wifi,num_wifi,num_connect
0,u_7869,s_461089,2017-08-06 12:20,121.962792,31.652743,b_36138751|-80|false;b_51989743|-47|false;b_40...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,-68.8,10,0
1,u_8234,s_70790,2017-08-06 16:50,121.963294,31.652419,b_309850|-79|false;b_27714427|-69|false;b_5716...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-58.1,10,0
2,u_11776,s_297192,2017-08-15 20:40,121.963307,31.653268,b_33826121|-60|false;b_33826070|-48|false;b_33...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.0,NaN,-54.4,10,0
3,u_11776,s_388487,2017-08-15 16:00,121.963730,31.653825,b_30034987|-50|false;b_33936609|-49|false;b_16...,m_1293,NaN,NaN,NaN,...,6.0,2.0,1.0,NaN,NaN,33.0,1.0,-59.3,10,0
4,u_27149,s_757418,2017-08-27 18:20,121.964003,31.654595,b_1025196|-71|false;b_54897624|-51|false;b_844...,m_1293,NaN,NaN,NaN,...,2652.0,90.0,NaN,NaN,NaN,NaN,10.0,-59.2,10,1


In [17]:
#train1 = pd.concat([train1,pd.DataFrame(l),pd.DataFrame(ll).pd.DataFrame(lll)], axis=1)
#train1 = pd.concat([train1,pd.DataFrame(lll)], axis=1)
#     one_hot_time_stamp = pd.get_dummies(train1['time_stamp'],prefix='time_stamp1')
#     train1 = pd.concat([train1,one_hot_time_stamp],axis=1)
# lb = preprocessing.LabelEncoder()
# train1["situation"] = list(map(lambda x,y:geohash2.encode(x,y,precision=9),train1["longitude"],train1["latitude"]))
# train1["situation1"] = lb.fit_transform(train1["situation"])
#     one_hot_situation = pd.get_dummies(train1['situation'],prefix='situation1')
#train1 = pd.concat([train1,one_hot_situation], axis=1)
len_ = train1.shape[0]
df_train = train1[:int(0.9 * len_)]
df_test=train1[int(0.9 * len_):]
lbl = preprocessing.LabelEncoder()
lbl.fit(list(df_train['shop_id'].values))
df_train['label'] = lbl.transform(list(df_train['shop_id'].values))
df_test['label'] = lbl.transform(list(df_test['shop_id'].values))  
num_class=df_train['label'].max()+1
feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','mall_id','wifi_infos',"situation","time_stamp"]]

c:\users\roy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
c:\users\roy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


# list(tmp_train["bssid"].unique())

In [10]:
feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','mall_id','wifi_infos',"situation","time_stamp"]]

In [21]:
df_train

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,mall_id,b_1003266,b_1003267,b_1003268,...,s_774697,s_819780,s_848684,s_97281,s_97701,avg_wifi,num_wifi,num_connect,avg_wifi,label
0,u_7869,s_461089,2017-08-06 12:20,121.962792,31.652743,b_36138751|-80|false;b_51989743|-47|false;b_40...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-68.800000,10,0,-68.800000,74
1,u_8234,s_70790,2017-08-06 16:50,121.963294,31.652419,b_309850|-79|false;b_27714427|-69|false;b_5716...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-58.100000,10,0,-58.100000,96
2,u_11776,s_297192,2017-08-15 20:40,121.963307,31.653268,b_33826121|-60|false;b_33826070|-48|false;b_33...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,-54.400000,10,0,-54.400000,27
3,u_11776,s_388487,2017-08-15 16:00,121.963730,31.653825,b_30034987|-50|false;b_33936609|-49|false;b_16...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,-59.300000,10,0,-59.300000,45
4,u_27149,s_757418,2017-08-27 18:20,121.964003,31.654595,b_1025196|-71|false;b_54897624|-51|false;b_844...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-59.200000,10,1,-59.200000,101
5,u_31313,s_3407316,2017-08-28 12:40,121.963232,31.652804,b_30792752|-91|false;b_30792753|-69|false;b_15...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-69.200000,10,0,-69.200000,33
6,u_32214,s_3628861,2017-08-30 17:20,121.963987,31.654461,b_28173341|-56|false;b_36772626|-55|false;b_33...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-65.000000,10,0,-65.000000,36
7,u_32331,s_3992936,2017-08-29 11:00,121.964086,31.654809,b_40972444|-61|false;b_26247857|-56|false;b_38...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-69.285714,7,1,-69.285714,67
8,u_32331,s_7466,2017-08-29 16:00,121.962699,31.652282,b_34372014|-64|false;b_11000487|-65|false;b_18...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-67.900000,10,1,-67.900000,99
9,u_32331,s_1126,2017-08-21 13:10,121.963402,31.652887,b_15389699|-77|false;b_2967078|-50|false;b_263...,m_1293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-76.400000,10,1,-76.400000,1


In [16]:
params = {
            'objective': 'multi:softmax',
            'eta': 0.1,
            'max_depth': 16,
            'eval_metric': 'merror',
            'seed': 1472,
            'missing': -999,
            'num_class':num_class,
            'silent' : 1,
            }
feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','mall_id','wifi_infos',"situation","Unnamed: 0","situation1","time_stamp"]] 

In [17]:
xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
xgbtest = xgb.DMatrix(df_test[feature])
watchlist = [ (xgbtrain,'train'), (xgbtrain, 'test') ]
num_rounds=60
model = xgb.train(params, xgbtrain, num_rounds,watchlist,early_stopping_rounds=15)
pre_=model.predict(xgbtest)
print("score:")
print(sum(pre_ == df_test['label'].values)*1.0 / len(df_test['label'].values))

[0]	train-merror:0.188909	test-merror:0.188909
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.16527	test-merror:0.16527
[2]	train-merror:0.150309	test-merror:0.150309
[3]	train-merror:0.137692	test-merror:0.137692
[4]	train-merror:0.126172	test-merror:0.126172
[5]	train-merror:0.115101	test-merror:0.115101
[6]	train-merror:0.106523	test-merror:0.106523
[7]	train-merror:0.098594	test-merror:0.098594
[8]	train-merror:0.088969	test-merror:0.088969
[9]	train-merror:0.081588	test-merror:0.081588
[10]	train-merror:0.075603	test-merror:0.075603
[11]	train-merror:0.069619	test-merror:0.069619
[12]	train-merror:0.063834	test-merror:0.063834
[13]	train-merror:0.059296	test-merror:0.059296
[14]	train-merror:0.054957	test-merror:0.054957
[15]	train-merror:0.051466	test-merror:0.051466
[16]	train-merror:0.047726	test-merror:0.047726
[17]	train-merror:0.044534	test-merror:0.044534
[

In [139]:
0.833482944345
0.834829443447
0.837522441652
0.840215439856
"situation1","time_stamp"
0.83842010772
0.841113105925
0.84066427289
0.841561938959

In [144]:
ExtC = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features=1, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
model = ExtC.fit(df_train[feature], df_train['label'])
pre_ = model.predict(df_test[feature])
print("score:")
print(sum(pre_ == df_test['label'].values)*1.0 / len(df_test['label'].values))

score:
0.784560143627


In [145]:
MLP_best = MLPClassifier()
model = MLP_best.fit(df_train[feature], df_train['label'])
pre_ = model.predict(df_test[feature])
print("score:")
print(sum(pre_ == df_test['label'].values)*1.0 / len(df_test['label'].values))

score:
0.728456014363


In [48]:
pd.DataFrame(l)

,b_10020256,b_10027728,b_1003266,b_1003267,b_1003268,b_1003269,b_10037764,b_1005773,b_1005774,b_1005873,...,b_978531,b_978532,b_978533,b_978534,b_9921200,b_9926206,b_9982663,b_9997036,b_9997654,b_9997685
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-73.0,-73.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
mall_list=list(set(list(shop.mall_id)))

In [38]:
result=pd.DataFrame()

In [55]:
train1=train[train.mall_id=="m_6587"].reset_index(drop=True)       
l=[]
wifi_dict = {}

In [57]:
for index,row in train1.iterrows():
    wifi_list = [wifi.split('|') for wifi in row['wifi_infos'].split(';')]
    for i in wifi_list:
        row[i[0]]=int(i[1])
        if i[0] not in wifi_dict:
            wifi_dict[i[0]]=1
        else:
            wifi_dict[i[0]]+=1
    l.append(row)    
delate_wifi=[]

In [58]:
for i in wifi_dict:
    if wifi_dict[i]<20:
        delate_wifi.append(i)
m=[]
for row in l:
    new={}
    for n in row.keys():
        if n not in delate_wifi:
            new[n]=row[n]
    m.append(new)

In [59]:
m

[{'b_26250516': -77,
  'b_26250517': -76,
  'b_26250578': -72,
  'b_26250579': -73,
  'b_26250580': -64,
  'b_26250581': -64,
  'b_26250593': -81,
  'b_29510856': -80,
  'b_30424471': -60,
  'b_52934996': -79,
  'latitude': 31.197415999999997,
  'longitude': 121.134451,
  'mall_id': 'm_6587',
  'shop_id': 's_609470',
  'time_stamp': Timestamp('2017-08-13 12:30:00'),
  'user_id': 'u_1158',
  'wifi_infos': 'b_26250579|-73|false;b_26250580|-64|false;b_26250581|-64|false;b_26250516|-77|false;b_52934996|-79|false;b_26250593|-81|false;b_26250517|-76|false;b_30424471|-60|false;b_26250578|-72|false;b_29510856|-80|false'},
 {'b_26250518': -76,
  'b_26250519': -78,
  'b_26250578': -74,
  'b_26250579': -73,
  'b_26250592': -85,
  'b_26250700': -75,
  'b_26250701': -74,
  'b_30424471': -51,
  'latitude': 31.197427,
  'longitude': 121.134354,
  'mall_id': 'm_6587',
  'shop_id': 's_609470',
  'time_stamp': Timestamp('2017-08-28 18:50:00'),
  'user_id': 'u_29096',
  'wifi_infos': 'b_30424471|-51|true

In [60]:
train1=pd.DataFrame(m)

In [85]:
train1.to_csv("experiment.csv")

In [27]:
#df_train=train1[train1.shop_id.notnull()]

In [29]:
#df_test=train1[train1.shop_id.isnull()]

In [62]:
total_len = train1.shape[0]

In [67]:
df_train = train1[:int(0.9 * total_len)]
df_test=train1[int(0.9 * total_len):]

In [68]:
lbl = preprocessing.LabelEncoder()
lbl.fit(list(df_train['shop_id'].values))
df_train['label'] = lbl.transform(list(df_train['shop_id'].values))
df_test['label'] = lbl.transform(list(df_test['shop_id'].values))
num_class=df_train['label'].max()+1

c:\users\roy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\roy\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [69]:
params = {
            'objective': 'multi:softmax',
            'eta': 0.1,
            'max_depth': 9,
            'eval_metric': 'merror',
            'seed': 0,
            'missing': -999,
            'num_class':num_class,
            'silent' : 1
            }
feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','time_stamp','mall_id','wifi_infos']]

In [78]:
ada_best = AdaBoostClassifier(algorithm='SAMME.R',
              base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, random_state=None,
              splitter='random'),
           learning_rate=1.5, n_estimators=5, random_state=7)

ExtC_best = ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features=1, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

DT_best = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

RFC_best = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=1, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=3,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

GBC_best = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

MLP_best = MLPClassifier()

In [70]:
xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
xgbtest = xgb.DMatrix(df_test[feature])
watchlist = [ (xgbtrain,'train'), (xgbtrain, 'test') ]
num_rounds=60

In [84]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),
    ('dt', DT_best), ('adac',ada_best),('gbc',GBC_best),('mlp',MLP_best)], voting='soft', n_jobs=3)
votingC = votingC.fit(df_train[feature],df_train['label'])
pre_ = votingC.predict(df_test[feature])

MaybeEncodingError: Error sending result: '[RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=1, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=3,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)]'. Reason: 'MemoryError()'

In [71]:
model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=15)

[0]	train-merror:0.115237	test-merror:0.115237
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.085849	test-merror:0.085849
[2]	train-merror:0.072847	test-merror:0.072847
[3]	train-merror:0.066435	test-merror:0.066435
[4]	train-merror:0.060468	test-merror:0.060468
[5]	train-merror:0.054502	test-merror:0.054502
[6]	train-merror:0.051118	test-merror:0.051118
[7]	train-merror:0.046398	test-merror:0.046398
[8]	train-merror:0.044171	test-merror:0.044171
[9]	train-merror:0.042123	test-merror:0.042123
[10]	train-merror:0.040253	test-merror:0.040253
[11]	train-merror:0.037492	test-merror:0.037492
[12]	train-merror:0.035889	test-merror:0.035889
[13]	train-merror:0.034464	test-merror:0.034464
[14]	train-merror:0.032327	test-merror:0.032327
[15]	train-merror:0.030457	test-merror:0.030457
[16]	train-merror:0.028854	test-merror:0.028854
[17]	train-merror:0.028141	test-merror:0.028141

In [72]:
pre_=model.predict(xgbtest)

In [73]:
print(sum(pre_ == df_test['label'].values)*1.0 / len(df_test['label'].values))

0.929487179487


In [ ]:
r=df_test[['row_id','shop_id']]
result=pd.concat([result,r])
result['row_id']=result['row_id'].astype('int')
result.to_csv(path+'sub.csv',index=False)